In [1]:
import pandas as pd
import seaborn as sns

In [2]:
metadata = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/hmp2_metadata_2018-08-20.csv')

/var/folders/sm/rgfxp6g94f18340p2rvqnsc00000gn/T/ipykernel_9623/2657062766.py:1: DtypeWarning: Columns (12,13,14,40,46,55,65,66,67,68,177,294,298,299,300,301,304,305,306,307,308,309,310,318,320,328,330,346,348,350,352,354,359,361,363,366,373,377,381,383,418,419,432,433,434,435,436,438,439,440,441,442,444,445,446,447,448,450,451,452,453,454) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/hmp2_metadata_2018-08-20.csv')


In [3]:
rnaseq = metadata[metadata['data_type'] == 'host_transcriptomics']

# display(rnaseq)

In [4]:
metagenomics = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/hmp2_metagenomics_metadata.csv')

In [5]:
display(metagenomics)

,Project,External ID,Participant ID,site_sub_coll,data_type,week_num,date_of_receipt,interval_days,visit_num,Research Project,...,FecalCal received at MGH:,Proteomics received at LBNL:,Stool Sample ID: Tube A (EtOH),Sample ID: Tube B (No preservative),Tube A and B received at Broad:,stool_id,smoking status,Number years smoked,Age when started smoking,How many cigarettes/cigars/etc. do you smoke per day?
0,G79889,CSM5FZ3N_P,C3001,C3001C1,metagenomics,0.0,2014-03-14,0.0,4,ibdmdb,...,No,No,SM-5FZ3N,SM-5FZ3P,No,NaN,NaN,NaN,NaN,NaN
1,G79894,CSM5FZ3R_P,C3001,C3001C2,metagenomics,2.0,2014-03-28,14.0,5,ibdmdb,...,No,No,SM-5FZ3R,SM-5UJBB,Yes,NaN,NaN,NaN,NaN,NaN
2,G79903,CSM5YRY7_P,C3001,C3001C3,metagenomics,4.0,2014-04-15,18.0,6,ibdmdb,...,No,No,SM-5YRY7,SM-5YRY8,Yes,NaN,NaN,NaN,NaN,NaN
3,G79913,CSM5FZ3V_P,C3001,C3001C4,metagenomics,6.0,2014-04-28,13.0,7,ibdmdb,...,No,No,SM-5FZ3V,SM-5FZ3W,Yes,NaN,NaN,NaN,NaN,NaN
4,G79926,CSM5FZ4C_P,C3001,C3001C5,metagenomics,8.0,2014-05-09,11.0,8,ibdmdb,...,No,No,SM-5FZ4C,SM-5FZ4D,Yes,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,G111931,PSMA26A1,P6038,P6038C12,metagenomics,24.0,2016-04-28,14.0,16,ibdmdb,...,No,No,SM-A26A1,SM-A26A2,No,NaN,NaN,NaN,NaN,NaN
1634,G110660,PSMA26A3,P6038,P6038C13,metagenomics,26.0,2016-05-12,14.0,18,ibdmdb,...,No,No,SM-A26A3,SM-A26A4,No,NaN,NaN,NaN,NaN,NaN
1635,G111932,PSMB4MC1,P6038,P6038C17,metagenomics,34.0,2016-07-08,8.0,22,ibdmdb,...,No,No,SM-B4MC1,SM-B4MC2,No,NaN,NaN,NaN,NaN,NaN
1636,G111934,PSMB4MC3,P6038,P6038C18,metagenomics,35.0,2016-07-20,12.0,23,ibdmdb,...,No,No,SM-B4MC3,SM-B4MC4,No,NaN,NaN,NaN,NaN,NaN


In [6]:
# Filter df1 based on the Participant ID column in df2
metagenomics_filt = metagenomics[metagenomics['Participant ID'].isin(rnaseq['Participant ID'])]

# print(metagenomics_filt['Participant ID'].unique())

In [7]:
rna_subset = rnaseq[['Participant ID', 'week_num']]
display(rna_subset)

,Participant ID,week_num
270,C3002,0.0
271,C3002,0.0
272,C3002,0.0
273,C3002,0.0
274,C3003,1.0
...,...,...
517,P6024,2.0
518,P6025,0.0
519,P6025,0.0
520,P6028,2.0


In [8]:
rna_group = rna_subset.groupby('Participant ID').max()

In [9]:
rna_group = pd.DataFrame(rna_group)

In [10]:
display(rna_group)

,week_num
Participant ID,
C3002,0.0
C3003,1.0
C3004,0.0
C3005,0.0
C3006,0.0
...,...
P6017,0.0
P6018,0.0
P6024,2.0


In [19]:
def find_closest_week(participant_id, week_num, metagenomic_df):
    # Filter the metagenomic_df for the same Participant ID
    participant_metagenomic = metagenomic_df[metagenomic_df["Participant ID"] == participant_id]
    
    # If no metagenomic samples for this participant, return None
    if participant_metagenomic.empty:
        return None
    
    # Calculate the difference between RNAseq week_num and metagenomic week_num
    participant_metagenomic["diff"] = participant_metagenomic["week_num"] - week_num

    print(participant_metagenomic['Participan ID', 'diff'])

    # # Select the sample with the smallest non-negative difference (closest week >= RNAseq week)
    # after_or_same = participant_metagenomic[participant_metagenomic["diff"] >= 0].sort_values(by="diff")
    # if not after_or_same.empty:
    #     return after_or_same.iloc[0]
    
    # # If no samples are found, select the sample with the smallest negative difference (closest week < RNAseq week)
    # before = participant_metagenomic[participant_metagenomic["diff"] < 0].sort_values(by="diff", ascending=False)
    # if not before.empty:
    #     return before.iloc[0]
    
    # # If no match is found, return None
    # return None


In [ ]:
for index, row in rna_group.iterrows():
    print(row['week_num'])

    find_closest_week(index, row['week_num'], metagenomics_filt)

In [ ]:
# Initialize an empty list to store the matched rows
matched_rows = []

# Iterate over each row in the RNAseq DataFrame
for index, row in rna_group.iterrows():
    participant_id = row.name  # Assuming 'Participant ID' is the index
    week_num = row['week_num']
    
    # Find the closest metagenomic sample for this participant
    matched_row = find_closest_week(participant_id, week_num, metagenomics_filt)
    
    # If a match is found, append it to the list of matched rows
    if matched_row is not None:
        matched_rows.append(matched_row)

# Convert the list of matched rows into a new DataFrame
matched_df = pd.DataFrame(matched_rows)

# Merge the RNAseq DataFrame with the matched metagenomic samples
merged_df = rna_group.merge(matched_df, on="Participant ID", suffixes=('_rna', '_metagenomic'))

# Print or use the resulting DataFrame
display(merged_df)

In [ ]:
merged_df = pd.merge(rnaseq, metagenomics_filt, on='Participant ID', how='left', suffixes=('_rnaseq', '_metagenomics'))

display(merged_df)

In [ ]:
viewing = merged_df[['Participant ID', 'week_num_rnaseq', 'week_num_metagenomics', 'Project_metagenomics']]

display(viewing)

In [ ]:
# Filter for the closest timepoint or earliest if none is before or at the RNA-seq timepoint
def find_closest_timepoint(group):
    # Filter for metagenomic samples that are before or at the RNA-seq timepoint
    valid_samples = group[group['week_num_metagenomics'] <= group['week_num_rnaseq'].iloc[0]]
    
    if not valid_samples.empty:
        # Find the sample with the closest (maximum) timepoint
        closest_sample = valid_samples.loc[valid_samples['week_num_metagenomics'].idxmax()]
    else:
        # If no valid sample exists, return the earliest metagenomic sample
        closest_sample = group.loc[group['week_num_metagenomics'].idxmin()]
    
    return pd.Series([closest_sample['External ID_metagenomics'], closest_sample['week_num_metagenomics']], 
                     index=['External ID_metagenomics', 'week_num_metagenomics'])

In [ ]:
closest_samples = merged_df.groupby(['Participant ID', 'Project_rnaseq']).apply(find_closest_timepoint).reset_index()

In [ ]:
rnaseq = rnaseq.rename(columns={'Project': 'Project_rnaseq'})

In [ ]:
# Merge the closest samples back to the original RNA-seq DataFrame
final_df = pd.merge(rnaseq, closest_samples, on=['Participant ID', 'Project_rnaseq'], how='left')

In [ ]:
final_df['External ID_metagenomics']

In [ ]:
final_df.to_csv('/Volumes/PGH-Backup/ibd_data/metadata/matched_rnaseq_mgx_sample_mapping.csv', index=False)

In [ ]:
for_viewing = final_df[['Participant ID', 'External ID', 'External ID_metagenomics', 'week_num', 'week_num_metagenomics', 'Project_rnaseq']]

display(for_viewing)

In [ ]:
for_viewing.to_csv('/Volumes/PGH-Backup/ibd_data/metadata/matched_rnaseq_mgx_sample_mapping.csv', index=False)

**Use Mapping File to Join MGX and RNA Data for Appropriate Samples**

In [ ]:
# Load datasets
mgx = pd.read_csv('/Volumes/PGH-Backup/ibd_data/humann_second_run/ibd_genefamilies_relab_clustered.tsv', sep='\t')
rna = pd.read_csv('/Volumes/PGH-Backup/ibd_data/rnaseq/tmm_normalized_counts.tsv', sep='\t')
mapping = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/matched_rnaseq_mgx_sample_mapping.csv')

In [ ]:
# Search for a value in a specific column and return the corresponding row
row = mgx.loc[mgx['sample_id'] == 'CSM5FZ3T']

# Display the row
print(row)

In [ ]:
# Tanspose so that Genes are columns and samples are rows in RNAseq data
rna = rna.set_index('Gene').T

rna.shape

In [ ]:
display(mgx)
display(rna)
display(mapping)

In [ ]:
rna = rna.reset_index().rename(columns={'index': 'sample_id'})
display(rna.head())

In [ ]:
# Create list of enriched NOD2 genes
with open ('/Volumes/PGH-Backup/ibd_data/rnaseq/GSE22611_NOD2_VS_CTRL_TRANSDUCED_HEK293T_CELL_UP.v2024.1.Hs.grp', 'r') as f:
    gene_list = [line.strip() for line in f.readlines()]
    gene_list = gene_list[2:]

print(gene_list)

In [ ]:
# Subset for genes present in table
present_genes = [gene for gene in gene_list if gene in rna.columns]

present_genes.append('sample_id')

In [ ]:
# Subset rnaseq data
rna_filt = rna[present_genes]

display(rna_filt)

In [ ]:
# # write out test subset rnaseq file:
# rna_filt.to_csv('/Volumes/PGH-Backup/ibd_data/rnaseq/test_subset_rnaseq.tsv', index=False, sep='\t')

In [ ]:
merged_mgx = pd.merge(rna_filt, mapping, left_on='sample_id', right_on='External ID', how='inner')

display(merged_mgx)

In [ ]:
merged_mgx['External ID_metagenomics'] = merged_mgx['External ID_metagenomics'].str.replace('_P', '')

In [ ]:
merged_mgx_rna = pd.merge(merged_mgx, mgx, left_on='External ID_metagenomics', right_on= 'sample_id', how='left')

display(merged_mgx_rna)

In [ ]:
subset_df = merged_mgx_rna[['sample_id_x', 'sample_id_y', 'External ID_metagenomics']]

display(subset_df)

In [ ]:
merged_mgx_rna.to_csv('/Volumes/PGH-Backup/ibd_data/rnaseq/test_subset_rnaseq_mgx.tsv', index=False, sep='\t')

In [ ]:
view = pd.read_csv('/Volumes/PGH-Backup/ibd_data/rnaseq/test_subset_rnaseq_mgx.tsv', sep='\t')

display(view)